In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import pickle as pk

X = pk.load(open('x_f-9_smot','rb'))
y =  pk.load(open('y_f-9_smot','rb'))

test_data = pd.read_csv('Test.csv')


In [12]:
feature_columns = ['grade_A_Component_1','max_luminosity','thickness',
                   'xmin','xmax','ymin','ymax','pixel_area',
                   'Component' ]
test_data['Component'] = test_data['x_component_1'] + test_data['x_component_2'] + \
                         test_data['x_component_3'] + test_data['x_component_4'] + \
                         test_data['x_component_5']  

test_data = test_data[feature_columns]

In [16]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss,f1_score,accuracy_score,roc_auc_score



from scipy.optimize import fmin 

err_score = []
test_pred_lgb = []

for i in range(10):
    
    err_lgb_ac = []
    err_lgb_f1 = []
    err_lgb_f1_c = []
    err_lgb_auc = []
    err_lgb_log_loss = []
    
    y_pred_tot_lgb = []
    y_pred_tot_lgb_c = []
    y_pred_tot_lgb_prob = []

    cutoff_value = []
    feature_importance_df = pd.DataFrame()

    fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=int(i*24))
    i = 1
    f = 0
    for train_index, test_index in fold.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        m=LGBMClassifier(random_state=int(i*12), scale_pos_weight=2)

        m.fit(X_train,y_train)

        y_pred = m.predict(X_test)
        y_pred_prob = m.predict_proba(X_test)[:,1]

        test_pred = m.predict(test_data)
        test_pred_proba = m.predict_proba(test_data)[:,1]


        err_ac = accuracy_score(y_test, y_pred)
        err_f1 = f1_score(y_test, y_pred)
        err_auc = roc_auc_score(y_test, y_pred_prob)
        err_logloss = log_loss(y_test,y_pred_prob)
        
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feature_columns
        fold_importance_df["importance"] = m.feature_importances_
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)


        f += 1
        print("Fold:", f)
        print("Accuracy:", err_ac)
        print('F1 Score:', err_f1)
        print("AUC Score:", err_auc)
        print('Log loss',err_logloss)
        print('\n')

        err_lgb_ac.append(err_ac)
        err_lgb_f1.append(err_f1)
        err_lgb_auc.append(err_auc)
        err_lgb_log_loss.append(err_logloss)

        y_pred_tot_lgb.append(test_pred)
        y_pred_tot_lgb_prob.append(test_pred_proba)
    
    err_score.append(np.mean(err_lgb_f1))
    test_pred_lgb.append(np.mean(y_pred_tot_lgb, 0).round().astype(int))
    
    print("Fold Score:", np.mean(err_lgb_f1))
    print()

Fold: 1
Accuracy: 0.8816901408450705
F1 Score: 0.8771929824561403
AUC Score: 0.9549609598171777
Log loss 0.31122323222807896


Fold: 2
Accuracy: 0.9070422535211268
F1 Score: 0.9014925373134328
AUC Score: 0.9669904145242176
Log loss 0.27782145921040957


Fold: 3
Accuracy: 0.9126760563380282
F1 Score: 0.906906906906907
AUC Score: 0.9743858312702343
Log loss 0.20860728570321113


Fold: 4
Accuracy: 0.8704225352112676
F1 Score: 0.8654970760233918
AUC Score: 0.942487145305656
Log loss 0.36052872126995655


Fold: 5
Accuracy: 0.8898305084745762
F1 Score: 0.8835820895522388
AUC Score: 0.9391617989721983
Log loss 0.3562678834985482


Fold Score: 0.8869343184504223

Fold: 1
Accuracy: 0.8901408450704226
F1 Score: 0.8842729970326411
AUC Score: 0.9517869612137371
Log loss 0.32772160957333146


Fold: 2
Accuracy: 0.8760563380281691
F1 Score: 0.8720930232558138
AUC Score: 0.9462959436297849
Log loss 0.3437283756192269


Fold: 3
Accuracy: 0.8901408450704226
F1 Score: 0.8828828828828829
AUC Score: 0.9528